In [1]:
import math
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Legend,Plot,GraphRenderer, Oval, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool,BoxZoomTool, Rect, WheelZoomTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes,from_networkx
from bokeh.palettes import Spectral4, Spectral8, Blues8, Greens8, magma,OrRd,Oranges, Viridis256, Oranges8
from bokeh.plotting import ColumnDataSource
from bokeh.models import ColumnDataSource, LabelSet, ColumnData,StaticLayoutProvider,GraphRenderer
from bokeh.layouts import gridplot
import numpy as np
from networkx.algorithms import approximation as approx
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
import csv

In [2]:
#Importing Nodes as Pandas DF
nodes = pd.read_csv("sheep_data/sheep_age.csv")
edges = pd.read_csv("sheep_data/sheep_edges.csv")
#Importing Edges as Tuples 
with open('sheep_data/sheep_edges.csv') as f:
    sheep_edges=[tuple(line) for line in csv.reader(f)]
sheep_edges.pop(0)

('source', 'target', 'weight')

In [3]:
#Creating the Network
X = nx.DiGraph()
X.add_weighted_edges_from(sheep_edges)

In [4]:
#Creating Features
src_dom = edges.groupby(['source']).sum()
tgt_dom = edges.groupby(['target']).sum()
src_dom = src_dom.reset_index()
tgt_dom = tgt_dom.reset_index()

In [5]:
df2 = pd.DataFrame([[1, 0, 0]],columns=['target','source','weight'])
tgt_dom = tgt_dom.append(df2)
tgt_dom = tgt_dom.sort_values('target')
tgt_dom = tgt_dom.drop(columns=['source'])
tgt_dom.rename(columns={'target': 'id', 'weight':'DominatedBy'}, inplace=True)

In [6]:
df2 = pd.DataFrame([[6, 0, 0]],columns=['source', 'target','weight'])
src_dom = src_dom.append(df2)
src_dom = src_dom.sort_values('source')
src_dom = src_dom.drop(columns=['target'])
src_dom.rename(columns={'source': 'id', 'weight':'DominatedOthers'}, inplace=True)                                                                  

In [7]:
#Merge Original Nodes DF to add Dominated
nodes = pd.merge(nodes, tgt_dom, how='inner', on='id')
nodes = pd.merge(nodes, src_dom, how='inner', on='id')
#1 - How many different sheep it dominated
src_diff = edges.groupby(['source']).sum()
#2 - How many different sheep it was dominated by
tgt_diff = edges.groupby(['target']).sum()
edges_work = edges.copy()
edges_work = edges_work.drop(columns=['weight'])
#del src
src = edges_work.groupby(['source']).count()
src = src.reset_index()
df2 = pd.DataFrame([[6, 0]],columns=['source', 'target'])
src = src.append(df2)
src = src.sort_values('source')
src.rename(columns={'source': 'id', 'target':'TargetsPerSource'}, inplace=True)
edges_work = edges.copy()
edges_work = edges_work.drop(columns=['weight'])
#del tgt
tgt = edges_work.groupby('target').count()
tgt = tgt.reset_index()
df2 = pd.DataFrame([[1, 0]],columns=['target', 'source'])
tgt = tgt.append(df2)
tgt = tgt.sort_values('target')
tgt.rename(columns={'target': 'id', 'source':'SourcesPerTarget'}, inplace=True)
nodes = pd.merge(nodes, src, how='inner', on='id')
nodes = pd.merge(nodes, tgt, how='inner', on='id')
#Load the node attributes
for j in nodes.columns:
    for i,k in zip(sorted(X.nodes()),pd.Series(nodes[j])):
        X.node[i][j] = k
#Create Edge Attributes:
color_series = []
for i in edges['weight']:
    if i>7:
        color_series.append(Spectral8[7])
    else:
        color_series.append(Spectral8[i])
edges['weight_color'] = color_series

In [8]:
nodes['DominatedOthers'] = nodes['DominatedOthers']+1
nodes['DominatedBy'] = nodes['DominatedBy']+1
nodes['TargetsPerSource'] = nodes['TargetsPerSource']+1
nodes['SourcesPerTarget'] = nodes['SourcesPerTarget']+1
nodes['DomBy/DomOthers'] = nodes['DominatedBy'].divide(nodes['DominatedOthers'])
nodes['DomBy/DomOthers'] = nodes['DomBy/DomOthers']+.1
nodes['Tgts/Srcs'] = nodes['TargetsPerSource'].divide(nodes['SourcesPerTarget'])
nodes['DominatedOthersPCGT'] = nodes['DominatedOthers']/nodes['DominatedOthers'].sum()
nodes['DominatedByPCGT'] = nodes['DominatedBy']/nodes['DominatedBy'].sum()
nodes['TotalInteractions'] = nodes['DominatedOthers']+nodes['DominatedBy']
nodes['WonPCGT'] = nodes['DominatedOthers']/nodes['TotalInteractions']
nodes['LostPCGT'] = nodes['DominatedBy']/nodes['TotalInteractions']
nds_dom_pt_lst = []
for i in nodes['DominatedOthersPCGT'].tolist():
    nds_dom_pt_lst.append(str(i))

In [9]:
print("A clique in an undirected graph G = (V, E) is a subset of the vertex set C subset e q V such that") 
print("for every two vertices in C there exists an edge connecting the two")
print("Max Clique: ", approx.max_clique(X))
mx_clique = approx.max_clique(X)

A clique in an undirected graph G = (V, E) is a subset of the vertex set C subset e q V such that
for every two vertices in C there exists an edge connecting the two
Max Clique:  {'4', '18', '24', '20', '26', '21'}


In [10]:
# or DiGraph, MultiGraph, MultiDiGraph, etc
mx_cliq = list(mx_clique)
H = nx.DiGraph(X.subgraph(mx_cliq))
type(H.edges())
H_edges_lst = []
for i in H.edges():
    H_edges_lst.append(i)

H_edges = pd.DataFrame(H_edges_lst, columns=['source', 'target'])
H_edges1 = pd.merge(edges, H_edges, how='right', left_on=['source','target'],right_on=['source','target'])
H_edges1=H_edges1.drop(columns=['weight_color'])
edge_weight_lst = []
for edge in edges['weight']:
    edge_weight_lst.append(str(edge))

In [11]:
color_series2 = []
for i in nodes['WonPCGT']:
    if i<0.2:
        color_series2.append(Oranges8[0])
    elif i<0.3:
        color_series2.append(Oranges8[1])
    elif i<0.4:
        color_series2.append(Oranges8[2])
    elif i<0.5:
        color_series2.append(Oranges8[3])
    elif i<0.6:
        color_series2.append(Oranges8[4])
    elif i<0.7:
        color_series2.append(Oranges8[5])
    elif i<0.8:
        color_series2.append(Oranges8[6])
    else:
        color_series2.append(Oranges8[7])
nodes['WonColor'] = color_series2

In [12]:
color_series3 = []
for i in 1/nodes['DomBy/DomOthers']:
    if i<0.5:
        color_series3.append(Blues8[0])
    elif i<1:
        color_series3.append(Blues8[1])
    elif i<1.5:
        color_series3.append(Blues8[2])
    elif i<2:
        color_series3.append(Blues8[3])
    elif i<2.5:
        color_series3.append(Blues8[4])
    elif i<3:
        color_series3.append(Blues8[5])
    elif i<3.5:
        color_series3.append(Blues8[6])
    else:
        color_series3.append(Blues8[7])
nodes['DomBy/DomOthers_color'] = color_series3

In [13]:
color_series4 = []
for i in nodes['Tgts/Srcs']:
    if i<1:
        color_series4.append(Greens8[0])
    elif i<2:
        color_series4.append(Greens8[1])
    elif i<3:
        color_series4.append(Greens8[2])
    elif i<4:
        color_series4.append(Greens8[3])
    elif i<6:
        color_series4.append(Greens8[4])
    elif i<8:
        color_series4.append(Greens8[5])
    elif i<9:
        color_series4.append(Greens8[6])
    else:
        color_series4.append(Greens8[7])
nodes['Tgts/Srcs_color'] = color_series4

In [30]:
#First Plot
N1 = len(X.nodes)
node_indices = list(range(N1))
nd_idx = [i+1 for i in node_indices]

p = figure(title="Graph Spectral Layout", x_range=(-1.1,1.1), y_range=(-1.1,1.1), toolbar_location="right")

p.title.text_color = "black"
p.title.text_font = "verdana"
p.title.text_font_style = "italic"
p.title.text_font_style = "bold"
p.background_fill_alpha = 0.07
p.title.align = "center"
p.title.text_font_size = '29pt'

p.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph_renderer = from_networkx(X,nx.spectral_layout, center=(0,0))

pos = nx.spectral_layout(X)
x,y=zip(*pos.values())

src = ColumnDataSource({'x':x,'y':y,'nodeid':nd_idx})
labels = LabelSet(x='x', y='y', text='nodeid', source=src)

graph_renderer.node_renderer.glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color",
                                            
                                            )

graph_renderer.node_renderer.selection_glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph_renderer.node_renderer.hover_glyph = Circle(size="size_dim", fill_alpha="dom_index", 
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph_renderer.node_renderer.data_source.data = dict(index=nd_idx, 
                                            size_dim = nodes['DominatedOthersPCGT']*600,
                                            dom_index = 30/nodes['DomBy/DomOthers'],
                                            dom_by = nodes['DominatedBy']/7,
                                            fi_color = nodes['DomBy/DomOthers_color'],
                                            age = nodes['age']/60
                                                    )

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width",line_join='round')
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width",line_join='round')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width",line_join='round')
graph_renderer.edge_renderer.data_source.data = dict(start=edges['source'],
                                                     end=edges['target'],
                                                     width=edges['weight'],
                                                     edge_col=edges['weight_color'],
                                                     width_alpha=edges['weight']/2,
                                                     edges_src=edges['source'],
                                                     edges_tgt=edges['target']
                                                    )

legend = Legend(items=[
                       ("Node Size: Count of Dominance by Percentage of All Sheep Interactions",[]),
                       ("Blue Intensity:  Ratio of Count of Times a sheep dominated vs. was dominated",[]),
                       ("Edge Size: Thicker Represents more Interactions between Sheep",[]),
                       ("Edge Color: Green Represents few, Yellow Medium, Red High Interactions Count",[]),
                       ("Edge Size: Thicker Represents more Interactions between Sheep",[]),
                        ("Circle Border: Thicker Represents more Interactions in which Sheep was Dominated",[]) 
                      ], location="top_right")

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

p.renderers.append(labels)
p.renderers.append(graph_renderer)
p.add_layout(legend)
show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='dbf0685b-aa83-48f1-ae36-aad826d20df6', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='20ce78b6-bf06-48cd-924a-b5a107e4678d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='5c217bbe-eac0-462b-a23d-3319e1a93185', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='4a730232-f260-4302-a7bf-fb54da6c4cc3', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='6d75572c-fe97-4286-af6f-8880bf81e72c', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='baae911f-379c-4877-9807-a

In [31]:
#First Plot
N1 = len(X.nodes)
node_indices = list(range(N1))
plot_options = dict(width=400, plot_height=400)
nds_dom_pt=nodes['DominatedOthersPCGT'].tolist()
nd_idx = [i+1 for i in node_indices]
#tgtdivsrcstgt-tgtdivsrc
node_src = dict(index=nd_idx, dom_oth_pcgt = nds_dom_pt,
                                    size_dim = nodes['DominatedOthersPCGT']*300,
                                    dom_index = 30/nodes['DomBy/DomOthers'],
                                    dom_by = nodes['DominatedBy']/7,
                                    fi_color = nodes['DomBy/DomOthers_color'],
                                    fi_color2 = nodes['Tgts/Srcs_color'],
                                    fi_color3 = nodes['WonColor'],
                                    age = nodes['age']/60,
                                    tgtdivsrcstgt=nodes['Tgts/Srcs']*30#,
                                    #tgtdivsrc = 1
                                
               )

edge_src = dict(start=edges['source'],end=edges['target'],
                                     width=edges['weight'],
                                     edge_col=edges['weight_color'],
                                     width_alpha=edges['weight']/2,
                                     edges_src=edges['source'],
                                     edges_tgt=edges['target'])

s1 = figure(title="Graph Layout Demonstration", x_range=(-1.1,1.1)
            , y_range=(-1.1,1.1),tools='pan,wheel_zoom,tap,xbox_select', toolbar_location="right", **plot_options)

s1.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph1 = from_networkx(X,nx.shell_layout, center=(0,0))

pos = nx.shell_layout(X)
x,y=zip(*pos.values())

src = ColumnDataSource({'x':x,'y':y,'nodeid':nd_idx})
labels = LabelSet(x='x', y='y', text='nodeid', source=src)
s1.title.text_color = "blue"
s1.title.text_font = "verdana"
s1.title.text_font_style = "italic"

graph1.node_renderer.glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color"
                                            
                                            )
graph1.node_renderer.selection_glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph1.node_renderer.hover_glyph = Circle(size="size_dim", fill_alpha="dom_index", 
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph1.node_renderer.data_source.data = node_src

graph1.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")

graph1.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph1.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph1.edge_renderer.data_source.data = edge_src

legend1 = Legend(items=[("Node ID",[])], location="top_right")


graph1.selection_policy = NodesAndLinkedEdges()
graph1.inspection_policy = EdgesAndLinkedNodes()

s1.renderers.append(graph1)

s1.renderers.append(labels)
s1.renderers.append(graph1)
s1.add_layout(legend1)

##Second Plot
N2 = len(X.nodes)
node_indices2 = list(range(N2))

s2 = figure(x_range=s1.x_range, y_range=s1.y_range,
               title="Second Graph",tools='pan,wheel_zoom,tap,xbox_select', 
                toolbar_location="right", **plot_options)

s2.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph2 = from_networkx(X,nx.shell_layout, center=(0,0))

s2.title.text_color = "blue"
s2.title.text_font = "verdana"
s2.title.text_font_style = "italic"                                    

graph2.node_renderer.glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2"
                                 )
graph2.node_renderer.selection_glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2"
                                            
                                            )

graph2.node_renderer.hover_glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2"
                                            
                                            )

graph2.node_renderer.data_source.data = node_src

graph2.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")

graph2.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph2.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph2.edge_renderer.data_source.data = edge_src


legend2 = Legend(items=[("Node ID",[])], location="top_right")

graph2.selection_policy = NodesAndLinkedEdges()
graph2.inspection_policy = EdgesAndLinkedNodes()

s2.renderers.append(graph2)

s2.renderers.append(labels)
s2.renderers.append(graph2)
s2.add_layout(legend2)

p = gridplot([[s1, s2]])

show(p)



E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='dbf0685b-aa83-48f1-ae36-aad826d20df6', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='20ce78b6-bf06-48cd-924a-b5a107e4678d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='5c217bbe-eac0-462b-a23d-3319e1a93185', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='4a730232-f260-4302-a7bf-fb54da6c4cc3', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='6d75572c-fe97-4286-af6f-8880bf81e72c', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='baae911f-379c-4877-9807-a

In [23]:
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import YlOrBr8,Oranges8

source = ColumnDataSource(dict(nodes))
color_mapper = LinearColorMapper(palette=Oranges8, low=nodes.WonPCGT.max(), high=nodes.WonPCGT.min())

tls="crosshair,pan,wheel_zoom,box_zoom,reset,hover,previewsave"

p = figure(x_axis_label='TargetsPerSource', y_axis_label='SourcesPerTarget', tools=tls, toolbar_location='above')
p.circle(x='TargetsPerSource', y='SourcesPerTarget', color={'field': 'WonPCGT', 'transform': color_mapper}, 
         size=20, alpha=0.6, source=source,legend="3*sin(x)")
hover =p.select(dict(type=HoverTool))
hover.tooltips = dict([
    ("Node ID", "@id"),
])

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='WonPCGT')

p.add_layout(color_bar, 'right')

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='dbf0685b-aa83-48f1-ae36-aad826d20df6', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='20ce78b6-bf06-48cd-924a-b5a107e4678d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='5c217bbe-eac0-462b-a23d-3319e1a93185', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='4a730232-f260-4302-a7bf-fb54da6c4cc3', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='6d75572c-fe97-4286-af6f-8880bf81e72c', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='baae911f-379c-4877-9807-a

In [21]:


p = figure(plot_width=300, plot_height=300)
p.multi_line(xs=[[1, 2, 3], [2, 3, 4]], ys=[[6, 7, 2], [4, 5, 7]],
             color=['red','green'], line_join='miter')
show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='dbf0685b-aa83-48f1-ae36-aad826d20df6', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='20ce78b6-bf06-48cd-924a-b5a107e4678d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='5c217bbe-eac0-462b-a23d-3319e1a93185', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='4a730232-f260-4302-a7bf-fb54da6c4cc3', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='6d75572c-fe97-4286-af6f-8880bf81e72c', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='baae911f-379c-4877-9807-a

In [27]:
from bokeh.models.annotations import Arrow
from bokeh.models.arrow_heads import OpenHead, NormalHead, VeeHead

p = figure(plot_width=600, plot_height=600)

p.circle(x=[0, 1, 0.5], y=[0, 0, 0.7], radius=0.1,
         color=["navy", "yellow", "red"], fill_alpha=0.1)

p.add_layout(Arrow(end=OpenHead(line_color="firebrick", line_width=4),
                   x_start=0, y_start=0, x_end=1, y_end=0))

p.add_layout(Arrow(end=NormalHead(fill_color="orange"),
                   x_start=[1,2], y_start=[0,2], x_end=[0.5,3], y_end=[0.7,3]))

p.add_layout(Arrow(end=VeeHead(size=35), line_color="red",
                   x_start=0.5, y_start=0.7, x_end=0, y_end=0))

show(p)

ValueError: expected an element of either String, Dict(Enum('expr', 'field', 'value', 'transform'), Either(String, Instance(Transform), Instance(Expression), Float)) or Float, got [1, 2]